In [1]:
import pandas as pd;
import xlrd;
import math;
import numpy as np;
import re;
import geopandas;
pd.set_option('display.max_colwidth', -1)

In [51]:
url = "http://www.nrc.uscg.mil/FOIAFiles/Current.xlsx"
xl = pd.ExcelFile(url)
#xl = pd.ExcelFile("current.xlsx")


In [70]:
url = "http://www.nrc.uscg.mil/FOIAFiles/Current.xlsx"

incidents = pd.read_excel(url,"INCIDENTS",dtype={'PIER_DOCK_NUMBER':str})


In [55]:
# Parse out sheets into separate DataFrames
calls = xl.parse("CALLS", na_values='');
incidents = pd.read_excel(url,"INCIDENTS",dtype={'PIER_DOCK_NUMBER':str})
incident_commons = xl.parse("INCIDENT_COMMONS");
incident_details = xl.parse("INCIDENT_DETAILS");
materials = xl.parse("MATERIAL_INVOLVED");
material_cr = xl.parse("MATERIAL_INV0LVED_CR");
trains = xl.parse("TRAINS_DETAIL");
traincars = xl.parse("DERAILED_UNITS");
vessels = xl.parse("VESSELS_DETAIL");
vehicles = xl.parse("MOBILE_DETAILS");

In [58]:
# Create a pretty date string for use in naming files later
import datetime
now = datetime.datetime.now()
now = now.strftime("%Y-%m-%d")

In [59]:
# Assemble all one-to-one tables into one big table
incident_commons = pd.merge(incident_commons,calls, on='SEQNOS')
incident_commons = pd.merge(incident_commons,incidents, on='SEQNOS')
incident_commons = pd.merge(incident_commons,incident_details, on='SEQNOS')
# Export many-to-one tables as separate files for import
materials.to_csv('materials' + now + '.csv')
material_cr.to_csv('material_cr' + now + '.csv')
trains.to_csv('trains' + now + '.csv')
traincars.to_csv('traincars' + now + '.csv')
vessels.to_csv('vessels' + now + '.csv')
vehicles.to_csv('vehicles' + now + '.csv')

## Skip this next cell if you're running full DB

In [ ]:
# Pull up our file showing the ID of the last record processed in the "Current.xls" file
# The file just keeps getting bigger every Sunday until calendar year end.
bookmark = pd.read_csv("bookmark")
# Drop all the old records that we've already processed
incident_commons = incident_commons[incident_commons.SEQNOS > int(bookmark.top[0])]

In [60]:
# Use full coordinates if available
incident_commons['new_latitude']  = incident_commons.LAT_DEG + (incident_commons.LAT_MIN / 60) + (incident_commons.LAT_SEC / 3600)
incident_commons['new_longitude'] = incident_commons.LONG_DEG + (incident_commons.LONG_MIN / 60) + (incident_commons.LONG_SEC / 3600)
incident_commons['new_latquad']   = incident_commons.LAT_QUAD
incident_commons['new_longquad']  = incident_commons.LONG_QUAD


In [61]:
# Basic coordinate cleaning function, from string to float
def splitclean(latitude):
    if isinstance(latitude,float):
        latitude = str(latitude)
    # Clear out extraneous characters
    latitude = re.sub(r'[A-Za-z]|\/|\'|\"|\&|\:|[\x00-\x1F\x80-\xFF]','',latitude)
    latitude = re.sub(r'\-',' ',latitude)
    latitude = re.sub(r' {2,}',' ',latitude)
    latitude = re.sub(r'\\p{C}','',latitude)
    latitude = re.sub(r'([0-9]{1,2}) (\..*)',r'\1' + r'\2',latitude)
    latitude = latitude.strip()
    # Break the string into components likely to be degrees, minutes and seconds
    components= latitude.split(' ')
    degrees = float; minutes = float; seconds = float;
    for i in range(len(components)):
        # No double decimal; ignore any digits after the second one
        components[i] = re.sub(r'\..*(\..*)',r'\0',components[i])
        # No leading and trailing zeroes
        components[i] = components[i].strip('0')
        # Only numeric characters and decimal points
        components[i] = re.sub(r'[^0-9.]+','',components[i])
        # No leading and trailing spaces
        components[i] = components[i].strip()
        # No leading and trailing decimal points
        components[i] = components[i].strip('.')
        if i < 2: # Weird case of single number with multiple decimal points
            components[i] = re.sub(r'\..*(\..*)',r'\0',components[i])
    if not components[0]:
        return # Skip if empty
    # Calculate degrees and minutes and compile into a single decimal unit
    else:
        degrees = float(components[0])
    if len(components) > 1:
        minutes = float(components[1]) / 60
        if len(components) == 3:
            seconds = float(components[2]) / 3600
            minutes = minutes + seconds
        if degrees > 0:
            degrees = degrees + minutes
        else:
            degrees = degrees - minutes
    return degrees

In [62]:
####################################
# Latitude from INCIDENT_LOCATION
####################################

# Start with boolean expression to operate on
null_latitude = (incident_commons.new_latitude.isnull()) & (incident_commons.INCIDENT_LOCATION.str.extract(r'^LAT(\/| ).*:(.*),')[1].notnull())

# Find existing substring to parse for degrees, hours, minutes
latitude = incident_commons.INCIDENT_LOCATION.str.extract(r'^LAT(\/| ).*:(.*)(N|S),')[1]
latquad  = incident_commons.INCIDENT_LOCATION.str.extract(r'^LAT(\/| ).*:(.*)(N|S),')[2]

# Clean the latitude
latitude = latitude.apply(splitclean)

incident_commons.loc[null_latitude,'new_latitude'] = latitude
incident_commons.loc[null_latitude,'new_latquad']  = latquad

####################################
# Longitude from INCIDENT_LOCATION
####################################

# Find existing substring to parse for degrees, hours, minutes
longitude = incident_commons.INCIDENT_LOCATION.str.extract(r'^LAT(\/| ).*:(.*)(N|S), (.*?)(W|E)')[3]
longquad  = incident_commons.INCIDENT_LOCATION.str.extract(r'^LAT(\/| ).*:(.*)(N|S), (.*?)(W|E)')[4]
    
# Clean the longitude
longitude = longitude.apply(splitclean)

incident_commons.loc[null_latitude,'new_longitude'] = longitude
incident_commons.loc[null_latitude,'new_longquad']  = longquad

####################################
# Latitude and longitude from INCIDENT_LOCATION using pattern of comma split
####################################

# Start with boolean expression to operate on
null_latitude = (incident_commons.new_latitude.isnull()) & (incident_commons.INCIDENT_LOCATION.str.extract(r'^([0-9].*),(.*)')[0].notnull())

# Find existing substring to parse for degrees, hours, minutes
latitude  = incident_commons.INCIDENT_LOCATION.str.extract(r'^([0-9].*),(.*)')[0]
longitude = incident_commons.INCIDENT_LOCATION.str.extract(r'^([0-9].*),(.*)')[1]
latquad   = latitude.str.extract(r'(N|S)')[0]
longquad  = longitude.str.extract(r'(W|E)')[0]

latitude  = latitude.apply(splitclean)
longitude = longitude.apply(splitclean)

incident_commons.loc[null_latitude,'new_latitude'] = latitude
incident_commons.loc[null_latitude,'new_latquad']  = latquad
incident_commons.loc[null_latitude,'new_longitude'] = longitude
incident_commons.loc[null_latitude,'new_longquad']  = longquad

####################################
# LATITUDE FROM LOCATION_ADDRESS
####################################

# Start with boolean expression to operate on
null_latitude = (incident_commons.new_latitude.isnull()) & (incident_commons.INCIDENT_LOCATION.str.extract(r'^LAT(\:| |\.)(.*)')[1].notnull())

# Find existing substring to parse for degrees, hours, minutes
latitude = incident_commons.LOCATION_ADDRESS.str.extract(r'^LAT(\:| |\.)(.*)')[1]
latquad  = latitude.str.extract(r'(N|S)')[0]

latitude  = latitude.apply(splitclean)

incident_commons.loc[null_latitude,'new_latitude'] = latitude
incident_commons.loc[null_latitude,'new_latquad']  = latquad

####################################
# LONGITUDE FROM LOCATION_ADDRESS
####################################

# Find existing substring to parse for degrees, hours, minutes
longitude = incident_commons.LOCATION_STREET1.str.extract(r'^LONG(\:| |\.)(.*)')[1]
longquad  = longitude.str.extract(r'(E|W)')

longitude  = latitude.apply(splitclean)

incident_commons.loc[null_latitude,'new_longitude'] = longitude
incident_commons.loc[null_latitude,'new_longquad']  = longquad

####################################
# Latitude and longitude from LOCATION_ADDRESS using pattern of comma split
####################################

# Start with boolean expression to operate on
null_latitude = (incident_commons.new_latitude.isnull()) & (incident_commons.LOCATION_ADDRESS.str.extract(r'^([0-9].*),(.*)')[0].notnull())

# Find existing substring to parse for degrees, hours, minutes
latitude  = incident_commons.LOCATION_ADDRESS.str.extract(r'^([0-9].*),(.*)')[0]
longitude = incident_commons.LOCATION_ADDRESS.str.extract(r'^([0-9].*),(.*)')[1]
latquad   = latitude.str.extract(r'(N|S)')[0]
longquad  = longitude.str.extract(r'(W|E)')[0]

latitude  = latitude.apply(splitclean)
longitude = longitude.apply(splitclean)

incident_commons.loc[null_latitude,'new_latitude'] = latitude
incident_commons.loc[null_latitude,'new_latquad']  = latquad
incident_commons.loc[null_latitude,'new_longitude'] = longitude
incident_commons.loc[null_latitude,'new_longquad']  = longquad


####################################
# LATITUDE FROM LOCATION_STREET1
####################################

# Start with boolean expression to operate on
null_latitude = (incident_commons.new_latitude.isnull()) & (incident_commons.LOCATION_STREET1.str.extract(r'^LAT(\:| |\.)(.*)')[1].notnull())

# Find existing substring to parse for degrees, hours, minutes
latitude = incident_commons.LOCATION_STREET1.str.extract(r'^LAT(\:| |\.)(.*)')[1]
latquad  = latitude.str.extract(r'(N|S)')[0]

latitude  = latitude.apply(splitclean)

incident_commons.loc[null_latitude,'new_latitude'] = latitude
incident_commons.loc[null_latitude,'new_latquad']  = latquad

####################################
# LONGITUDE FROM LOCATION_STREET1
####################################

# Find existing substring to parse for degrees, hours, minutes
longitude = incident_commons.LOCATION_STREET2.str.extract(r'^LONG(\:| |\.)(.*)')[1]
longquad  = longitude.str.extract(r'(E|W)')

longitude  = latitude.apply(splitclean)

incident_commons.loc[null_latitude,'new_longitude'] = longitude
incident_commons.loc[null_latitude,'new_longquad']  = longquad

####################################
# Latitude and longitude from LOCATION_STREET1 using pattern of comma split
####################################

# Start with boolean expression to operate on
null_latitude = (incident_commons.new_latitude.isnull()) & (incident_commons.LOCATION_STREET1.str.extract(r'^([0-9].*),(.*)')[0].notnull())

# Find existing substring to parse for degrees, hours, minutes
latitude  = incident_commons.LOCATION_STREET1.str.extract(r'^([0-9].*),(.*)')[0]
longitude = incident_commons.LOCATION_STREET1.str.extract(r'^([0-9].*),(.*)')[1]
latquad   = latitude.str.extract(r'(N|S)')[0]
longquad  = longitude.str.extract(r'(W|E)')[0]

latitude  = latitude.apply(splitclean)
longitude = longitude.apply(splitclean)

incident_commons.loc[null_latitude,'new_latitude'] = latitude
incident_commons.loc[null_latitude,'new_latquad']  = latquad
incident_commons.loc[null_latitude,'new_longitude'] = longitude
incident_commons.loc[null_latitude,'new_longquad']  = longquad

In [63]:
def findstreet(location):
    if isinstance(location,float):
        return False
    elif location == '':
        return False
    else:
        location = location.encode('utf-8')
        location = str(location)
        location = re.sub(r'\\p{C}|[\x00-\x1F\x80-\xFF]','',location)
        if re.search(r' (RD|ROAD|AVE|AVENUE|LN|LANE|PL|PLACE|HWY|HIGHWAY|BLVD|BOULEVARD|CT|COURT|CIR|CIRCLE|ROUTE|RTE|WY|WAY)( |\.|$)',location):
            return True
    return False
has_street = incident_commons.LOCATION_ADDRESS.apply(findstreet) == True

incident_commons['new_street'] = ''
incident_commons['new_street'] = np.nan
incident_commons.loc[has_street,'new_street'] = incident_commons.LOCATION_ADDRESS

has_street = (incident_commons.LOCATION_STREET1.apply(findstreet) == True) & (incident_commons.new_street.isnull())
incident_commons.loc[has_street,'new_street'] = incident_commons.LOCATION_STREET1

has_street = (incident_commons.INCIDENT_LOCATION.apply(findstreet) == True) & (incident_commons.new_street.isnull())
incident_commons.loc[has_street,'new_street'] = incident_commons.INCIDENT_LOCATION


In [64]:
geo = geopandas.GeoDataFrame(incident_commons)
counter = 0
from geopy.geocoders import Bing
bingkey = "AuNPKK6wEhtJOp2JSz1iQQwqgCptimUiyamkP18Bnz4ycjMaxcFdd1kYEqyWrdxL"

for row in geo.itertuples():
    counter+=1
    # Skip this record for geocoding if latitude is already populated
    if not math.isnan(row.new_latitude):
        continue
    # Compile the address using available street/city/county/state fields
    address = ''
    if not isinstance(row.LOCATION_STATE,float):
        address = str(row.LOCATION_STATE)
    if not isinstance(row.LOCATION_NEAREST_CITY,float):
        address = str(row.LOCATION_NEAREST_CITY) + ',' + address
    else:
        if not isinstance(row.LOCATION_COUNTY,float):
            address = str(row.LOCATION_COUNTY) + ' COUNTY,' + address
    if not isinstance(row.new_street,float):
        address = str(row.new_street) + ',' + address
    if address == '':
        if not isinstance(row.INCIDENT_LOCATION,float):
            address = str(row.INCIDENT_LOCATION,float)
        else:
            continue
    print str(counter) + ' ' + address
    # Geocode the address
    try:
        location = geopandas.tools.geocode(address,provider="Bing",api_key=bingkey)
        location['SEQNOS'] = row.SEQNOS
        if counter == 1:
            locations = geopandas.GeoDataFrame(location)
        else:
            locations = locations.append(location)
    except:
        continue
geo = geo.merge(locations,how='left',on='SEQNOS')

1 FARMINGDALE,NY
2 BLOUNTS CREEK,NC
3 SOUTH OF LAFAYETTE AVE STATION,BROOKLYN,NY
4 560 HWY 44,LAPLACE,LA
5 RAIL ROAD CROSSING,CAPAC,MI
6 PENDLETON,OR
7 MIDKIFF,TX
8 25 FAIRVIEW RD,WESTBROOK,CT
9 STONINGTON,CT
11 HONOLULU,HI
13 12222 PORT RD,PASADENA,TX
15 7524 KENSTEAD CIRCLE,CHARLOTTE,NC
17 2265 COUNTY ROAD 6,PIEDMONT,AL
18 250 INDUSTRIAL ROAD,PASCAGOULA,MS
21 1072 ALMONESSON ROAD,DEPTFORD,NJ
22 1 BENEDEK RD,PRINCETON,NJ
23 PEMBROKE,NH
25 GROTON,CT
26 1023 BAY VISTA COURT,GALT,CA
27 GENISON,MI
28 5281 SOUTHERN RR STATION RD,LINWOOD,NC
29 POMPANO,FL
31 GALT,CA
33 HOUSTON,TX
34 CORAL SPRINGS,FL
35 MIDLAND,TX
36 PENSACOLA,FL
37 SCOTTSDALE,AZ
38 LAKELAND,FL
39 ALPINE AVE AND WEST LANE,STOCKTON,CA
40 3001 MILWAUKEE RD,BELOIT,WI
41 TALLAPOOSA,GA
42 3314 RIVER RD,WILMINGTON,NC
44 96 W 11TH AVE,GLOVERSVILLE,NY
48 MARROWBONE,KY
49 WHITLEY CITY,KY
50 1751 LIVERMORE ROAD,HARTFORD,KY
51 19210 E HARDY RD,HOUSTON,TX
52 LIVE OAK,FL
53 #2 DUTCHESS AVE,POUGHKEEPSIE,NY
54 CHEROKEE FALLS,SC
55 ASHTABULA

361 615 DESTREHAN AVE,HARVEY,LA
362 2060 WESTERN AVE,GUILDERLAND,NY
363 BEALE AFB,CA
364 MARCUS HOOK,PA
365 MANDEVILLE,LA
366 FORT EUSTIS,VA
367 28 MANOR LANE,COPIAGUE,NY
368 CHESAPEAKE,VA
369 WESTPORT,WA
370 20775 POWER PLANT ROAD,OTTUMWA,IA
371 2350 MARINSHIP WAY,SAUSALITO,CA
372 NEWPORT BEACH,CA
373 LONG BEACH,CA
374 PENSACOLA,FL
375 FAIRVIEW AND SWANNOA RIVER ROAD,ASHVILLE,NC
376 LONG BEACH,CA
377 SR 279 JIM REESE RD,OAK HILL,OH
378 CALVERT CITY,KY
379 BRUNSWICK,ME
380 PEORIA,IL
381 ADAMS AVENUE & PRAIRIE STREET,OKOLONA,MS
382 SAN ANTONIO,TX
383 WEST CHICAGO,IL
385 PROGRESS RD AT HWY 385,COLLIERVILLE,TN
386 32435 W REICHMUTH RD,FREEMONT,NE
387 WEST HWY 160 AND HWY 529,COTTON VALLEY,LA
388 MONTGOMERY COUNTY,OH
389 PORT OCONNER,TX
390 260 SIMPSON HWY #149,MAGEE,MS
391 PORT OCONNER,TX
392 2 DUTCHESS AVE,POUGHKEEPSIE,NY
393 CHARLOTTE,NC
394 3822 DARLINGTON RD,NEWBURG,IN
395 SPRINGFIELD,OH
396 32435 W REICHMUTH RD,FREEMONT,NE
397 BOKCHITO,OK
398 CORPUS CHRISTI COUNTY,TX
399 CHATTANOOGA,

703 15602 JACINTO PORT BLVD.,HOUSTON,TX
704 FRANKLIN,LA
705 6176 JOHNS RD,HUEYTOWN,AL
706 1430 EVERGLAD ROAD,BROWNSVILLE,TX
707 231 ROAD 3168,HASTINGS,NE
708 7281 MOUNT HAMILTON RD,MOUNT HAMILTON,CA
709 ORANGE FIELD,TX
710 JONESPORT,ME
711 LOUISVILLE,KY
713 7049 HAMILTON AVE,PITTSBURGH,PA
715 HOUSTON,TX
716 OFF OF NEPTUNE BLVD.,LONG BEACH,NY
719 1180 NIMITZ AVE,VALLEJO,CA
720 WHITES BOTTOM ROAD,PINDERGRASS,GA
721 4162 S CREEK ROAD,CHATTANOOGA,TN
722 AVENTURA,FL
723 1480 SEWARD AVE,SITKA,AK
724 FULTON,TX
725 FULTON,TX
726 1 SW 1ST PLACE,CRYSTAL RIVER,FL
728 POWDERLY,TX
730 ORANGE,TX
731 OAK LAWN,IL
732 CHICAGO,IL
733 2397 CHESTNUT AVE,LONG BEACH,CA
735 WINNEMUCCA,NV
736 HAYWARD,CA
737 KETCHIKAN,AK
738 SPRINGFIELD,MO
739 ROANOKE,VA
740 PORT ARTHUR,TX
741 SILVERTON,OR
742 4500 WRANGLE HILL RD,DELAWARE CITY,DE
743 13800 EAST COUNTY RD 230,MIDLAND,TX
744 NORTH MIAMI BEACH,FL
745 901 LOVERAGE ROAD,PITTSBURG,CA
746 1010 SHOP RD,ST. PAUL,MN
747 1010 SHOP ROAD,ST PAUL,MN
748 HOBOKEN,NJ
749 SOUT

1063 SEEKONK,MA
1064 NORTH JACKSON,OH
1065 CORPUS CHRISTI,TX
1066 STATE LINE CLUB RD,GREENWOOD,LA
1067 CHICAGO,IL
1068 NASHVILLE,TN
1069 PORT TOWNSEND,WA
1070 TUPMAN,CA
1071 PORT ARTHUR,TX
1073 RIVERSIDE,MO
1074 RICKS RD,SELMA,NC
1075 CALUMET ROAD,CHESTERTON,IN
1077 CASPER,WY
1078 MAIN STREET AND CEDAR BAY ROAD,JACKSONVILLE,FL
1080 OLIVER SPRINGS,TN
1081 THE DALLES,OR
1082 12201 EAST PLEASANT LAKE RD,MANCHESTER,MI
1085 11950 CORKSCREW RD,ESTERO,FL
1086 MILLSBORO,DE
1087 13264 MOUNTAIN RD,GLEN ALLEN,VA
1088 JACKSONVILLE,FL
1089 ROANOKE,VA
1090 CARTERET,NJ
1091 NEODESHA,KS
1093 BLAIRSVILLE,PA
1095 PERRE HAUTE,IN
1096 45895 SMITHBERGER RD,SUMMERFIELD,OH
1097 WINDCHESTER,VA
1098 2551 SOUTH SHORE BLVD,LEAGUE CITY,TX
1102 PORT ARTHUR,TX
1103 25915 SE FRONTAGE RD.,CHANNAHON,IL
1104 CARSON,CA
1106 CLINTON,NC
1107 MIDKIFF,TX
1108 TACOMA,WA
1109 UNIVERSITY PLACE,WA
1110 9595 CARTERS MILL RD,PLAIN CITY,OH
1112 950 PORT WASHINGTON RD,GRAFTON,WI
1113 NEW HAVEN,CT
1114 2500 CEDARDALE RD,MOUNT VERNON

1409 HUDSON,OH
1410 NAPLES,FL
1411 HURRICANE,WV
1414 MIDKIFF,TX
1415 RANKIN,TX
1416 CHICAGO,IL
1418 13305 IRIS RD,NEOSHO,MO
1419 LITTLE FARMS AVE,KENNER,LA
1420 NORTHORD,CT
1421 7100 US HIGHWAY 90,DAPHNEY,AL
1422 MIDKIFF,TX
1425 NORTH EAST,MD
1426 MIDLAND,TX
1427 9502 BAYPORT BLVD,PASADENA,TX
1428 ROSEVILLE,CA
1429 BALTIMORE,MD
1430 ATHENS,GA
1431 63149 COLUMBIA RIVER HWY,DEER ISLAND,OR
1432 ARGO,IL
1433 RANKIN,TX
1434 300 S BURNHAMS BLVD,HAGERSTOWN,MD
1437 ST. MARY COUNTY,LA
1438 1322 AIRLIE ROAD,WILMINGTON,NC
1439 COLUMBIA,OR
1440 HUTCHINS,TX
1442 ELYS FERRY ROAD,LYME,CT
1445 DEEN RD,FT WORTH,TX
1447 INTERSECTION OF RAWSON RD & SHITTENDEN RD; BOOSTER BANK #85747,CORNING,CA
1448 COLUMBIA CITY,OR
1449 NASHVILLE,TN
1450 PECOS COUNTY,TX
1451 PENSACOLA,FL
1452 HOUSTON,TX
1453 HONOLULU,HI
1454 25 CAUSEWAY BLVD,CLEARWATER,FL
1455 LITTLE FALLS,MN
1456 MAN,WV
1457 4100 BLOCK OF STATE ROUTE 529,EVERETT,WA
1458 EL PASO,TX
1459 PHOENIX CITY,AL
1460 1964 OLD DUNBAR ROAD,WEST COLUMBIA,SC
1462 RIDG

...

4595 OMAHA,NE
4596 1900 SOUTH CONWAY ROAD,ORLANDO,FL
4597 7737 HAMPTON BLVD,NORFOLK,VA
4598 1420 NORTH WITTER RD,PASADENA,TX
4599 STOCKTON,CA
4600 CLAIRTON,PA
4601 CENTRE HALL,PA
4602 NORTH LIBERTY,IA
4603 1100 INDIANPOLIS BLVD,HAMMOND,IN
4604 1900-BLOCK FALLS ROAD,BALTIMORE,MD
4605 CAMARILLO,CA
4607 4401 HWY 108 SOUTH,SULPHUR,LA
4608 OCEANSIDE,CA
4609 BOULDER,CO
4610 SAN DIEGO,CA
4612 KITTERY,ME
4613 STATE LINE ROAD AND LONGSPUR ROAD,RESERVE,KS
4614 OLYMPIA,WA
4615 4300 HWY 108,WESTLAKE,LA
4616 700-BLOCK PARK AVENUE,KETCHIKAN,AK
4618 EAST BOSTON,MA
4620 PERU,IN
4621 1624 MARTINDALE AVENUE,WYOMING,MI
4622 PARIS,TN
4623 1610 E SEPULVEDA BLVD,CARSON,CA
4624 ROCKFORD,IL
4625 PEARL HARBOR,HI
4626 77870 HERMISTON ROAD,STANFIELD,OR
4627 77870 HERMISTON ROAD,STANFIELD,OR
4628 BERKLEY,IL
4629 3875 STATE ROUTE 65,LEIPSIC,OH
4630 2195 NORTH REDDING RD,DENVER,PA
4632 9724 32ND PLACE WEST,EVERETT,WA
4633 DECATUR,AL
4634 NORFOLK,VA
4635 CHARLESTON,SC
4636 VENICE,FL
4637 BETHPAGE,TN
4639 5834 WARD A

4962 AUGUSTA,GA
4963 BANKS,OR
4966 4991 ENKA HWY,MORRISTOWN,TN
4967 E AVE AND WINFIELD ST,NORWALK,CT
4969 DELRAY BEACH,FL
4970 SAN PEDRO,CA
4972 7397 N STATE ROUTE 7,CHESHIRE,OH
4973 ABERDEEN,MD
4974 OXNARD,CA
4976 ROANOKE,VA
4977 107TH AVE & BUCKEYE ROAD,AVONDALE,AZ
4978 NORFOLK,VA
4979 107 HARRIS MINING ROAD,SPRUCE PINE,NC
4980 COQUILLE,OR
4982 CHESAPEAKE CITY,VA
4983 DECATUR,IL
4984 WESTMORLAND,CA
4985 RISING SUN,IN
4986 FAIRFAX,VA
4987 LOWER MAKEFIELD TWP,PA
4988 222 KYSER BLVD,MADISON,AL
4989 10047 E WILSON RD,INDEPENDENCE,MO
4990 STATE ROAD 2,NEW MARTINSVILLE,WV
4992 BEND,OR
4994 HAMTRAMCK,MI
4995 HOUSTON,TX
4996 DRISCOLL,TX
4997 GREENSBORO,NC
4998 3215 COLLEGE RD,FAIRBANKS,AK
5001 NORTH CANTON,OH
5002 HOFFMAN ESTATES,IL
5003 WEBSTER,FL
5004 BRAWLEY,CA
5005 PORTERVILLE,CA
5007 HEPPNER,OR
5008 GLADSTONE,MI
5009 BIG SPRING,TX
5010 SILVER CREEK,GA
5011 NORWALK,CT
5012 BIRMINGHAM,AL
5013 INTERSECTION OF 112TH STREET AND 77TH AVENUE,PINECREST,FL
5014 BOSTON,MA
5016 FLEMING KEY,FL
5017

5334 701 HWY 7,EXCELSIOR,MN
5335 2700 BROENING HWY,BALTIMORE,MD
5336 7616 WEST DUTCH AVE,HESSTON,KS
5337 111 W MARINE WAY,KODIAK,AK
5338 ROGERS,NE
5339 LAKELAND,FL
5340 HOMER,PA
5341 3 RATNA COURT,NOTTINGHAM,MD
5342 1150 BALLENA BLVD,ALAMEDA,CA
5343 244 JOHN B BROOKS RD,TINDERGRASS,GA
5345 OSAGE COUNTY,OK
5346 MIDKIFF,TX
5347 1005 E WILL ROGERS BLVD,CLAREMORE,OK
5348 LAS VEGAS,NV
5349 OLD HOPLAND,CA
5350 OTTAWA,IL
5351 7959 HIGHWAY 178 WEST,BYHALIA,MS
5352 5900 HWY 225 EAST,DEER PARK,TX
5353 CORPUS CHRISTI,TX
5354 BILOXI,MS
5355 2510 WHITE HALL BLVD,WHITE HALL,WV
5356 FORT PIERCE,FL
5357 2651 BENTON RD,BOSSIER,LA
5358 LA PUSH,WA
5359 DUTCH HARBOR,AK
5360 SAULT STE. MARIE,MI
5361 MIDLAND,TX
5362 CHICAGO,IL
5363 MANDEVILLE,LA
5364 2200 FORT AMANDA ROAD,LIMA,OH
5365 PELLA,IA
5367 107 NEW COUNTY ROAD,THOMASTON,ME
5368 APTOS,CA
5370 1026 CABRAS HWY,PITI,GU
5371 ROCKINGHAM,NC
5372 WEST SACRAMENTO,CA
5373 LANCASTER,CA
5374 E. OLNEY AVE & WEST FORD RD,PHILADELPHIA,PA
5375 MIAMI BEACH,FL
5379 C

5688 CHICAGO,IL
5689 CHICAGO,IL
5690 HOUSTON,TX
5691 371 SOUTH HWY 59,MERCED,CA
5692 SAN DIEGO,CA
5693 CABO ROJO,PR
5694 SARNIA,CN
5695 24685 HIGHWAY 281 SOUTH,SAN ANTONIO,TX
5696 EL PASO,TX
5697 1402 NORTH COLLIER BLVD,MARKO ISLAND,FL
5698 FORT BRAGG,CA
5699 1050 BAILEY BLVD,CHULA VISTA,CA
5700 STATE HWY 25,GREENVILLE,SC
5701 PHILADELPHIA,PA
5702 LONG BEACH,CA
5703 SAN ANTONIO,TX
5704 4201 VICTORIA AVE,OXNARD,CA
5705 CARPENTERS BAYOU; INTERSECTION OF SHELDON RD AND CARPENTERS BAYOU,HARRIS COUNTY,TX
5707 29501 STATE ROUTE 7,STRATTON,OH
5708 BEVERLY ROAD,BROOK,NY
5709 TITUSVILLE,FL
5711 WB HWY 402 AT FRONT ST,SARNIA; COUNTY OF LAMBTON,CN
5712 WEST FARGO,ND
5713 15536 RIVER ROAD,NORCO,LA
5714 TOGIAK,AK
5716 5767 OLD HWY 61,ESKO,MN
5717 BATON ROUGE,LA
5718 SAND POINT,AK
5719 LAKE WORTH,FL
5720 WASILLA,AK
5722 40044 N.E. MILLER RD,WASHOUGAL,WA
5723 WASHINGTON,UT
5724 HOUSTON,TX
5725 965 N NIMITZ HIGHWAY,HONOLULU,HI
5726 SACRAMENTO COUNTY,CA
5727 INDIANAPOLIS,IN
5729 5032 DORT HIGHWAY,FLINT

6030 12459 JACKSON TRACE RD,LINCOLN,AL
6031 KITTERY,ME
6032 BARTLETT,TX
6033 1026 CABRAS HIGHWAY,PITI,GU
6034 FLAT ROCK,MI
6036 1801 N ROOSEVELT BLVD,KEY WEST,FL
6037 ISABEL,TX
6040 BIRGMINGHAM,AL
6042 FAIRHAVEN,MA
6043 25733 OLD HWY 112,CAMERON,OK
6044 FAJARDO,PR
6045 801 SEABREEZE BLVD,FORT LAUDERDALE,FL
6046 2125 S COUNTY RD 125 W,LOGANSPORT,IN
6048 2035 KEYSTONE BLVD,NORTH MIAMI,FL
6049 2698 HICKORY BLVD,HUDSON,NC
6050 PORTSMOUTH,VA
6052 PORTSMOUTH,VA
6053 LOS ANGELES,CA
6054 FREEMAN LN,MADISONVILLE,TN
6056 3919 18TH AVE W,SEATTLE,WA
6057 KAYENTA,AZ
6058 WHITE,GA
6060 FORT WALTON BEACH,FL
6061 HALF MILE SOUTH OF AVENUE 144 AND ROAD 200,PORTERVILLE,CA
6063 827 GULF RD.,SURFSIDE BEACH,TX
6064 1432 RIVER RD,MARYSVILLE,MI
6065 MONTEREY,CA
6066 MIDLAND,TX
6067 7001 SEAVIEW AVE NW,SEATTLE,WA
6068 SOMERSET,PA
6069 SEATTLE,WA
6070 ARABI,LA
6072 KEYSTONE ROAD HEADING WEST PAST AUSTIN ROAD,IMPERIAL,CA
6073 RED BANK,NJ
6074 BANNING,CA
6075 21575 AMES LANE,DEXTER CITY,OH
6076 SUB: NH ROUTE MAI

In [ ]:
# REDO merge without geocoding
#geo = geopandas.GeoDataFrame(incident_commons)
#geo = geo.merge(locations,how='left',on='SEQNOS')

In [65]:
# Create a text file noting record ID of where we left off with the last import of data.
biggest = geo.SEQNOS.max()
bookmark = pd.DataFrame({'top':[biggest]})
bookmark.to_csv('bookmark')

In [66]:
# Clean up latitudes and longitudes to proper sign based on hemisphere
geo.new_latitude[(geo.new_latquad == 'N')|geo.new_latquad.isnull()] = abs(geo.new_latitude)
geo.new_latitude[geo.new_latquad == 'S'] = -1 * abs(geo.new_latitude)
geo.new_longitude[geo.new_longquad == 'E'] = abs(geo.new_longitude)
geo.new_longitude[(geo.new_longquad == 'W')|geo.new_longquad.isnull()] = -1 * abs(geo.new_longitude)
#geo.sample(10)

# Create a geodataframe by converting coordinate data to Point objects
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(geo.new_longitude, geo.new_latitude)]
points   = geopandas.GeoDataFrame(geometry, geometry=geometry)

# Boolean to specify records to update using the coordinate data
null_coordinates = geo.geometry.isnull()
#null_coordinates = geo.new_latitude.notnull()

# Update the geometry with data from the latitude and longitude coordinates
geo.loc[null_coordinates,'geometry'] = points


#geo['INCIDENT_DATE_TIME'] = geo['INCIDENT_DATE_TIME'].str
#geo['DATE_TIME_RECEIVED'] = geo['DATE_TIME_RECEIVED'].astype(str)
#geo['DATE_TIME_COMPLETE'] = geo['DATE_TIME_COMPLETE'].astype(str)
#geo['CR_BEGIN_DATE']      = geo['CR_BEGIN_DATE'].astype(str)
#geo['CR_END_DATE']        = geo['CR_END_DATE'].astype(str)
#geo['CR_CHANGE_DATE']     = geo['CR_CHANGE_DATE'].astype(str)
#geo.info()



/Users/ssuo1/OneDrive - Advance Local Media LLC/Code/oil-spills/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ssuo1/OneDrive - Advance Local Media LLC/Code/oil-spills/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ssuo1/OneDrive - Advance Local Media LLC/Code/oil-spills/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http:/

In [68]:
# Output everything as a text file for use elswhere
jsonfile = 'spillcalls' + now + '.geojson'

geo.to_file(jsonfile, driver='GeoJSON')
